In [1]:
import pandas as pd
import numpy as np
import os, sys, shutil
import omegaconf
import matplotlib.pyplot as plt

from mbrl.util.plot_and_save_push_data import plot_and_save_push_plots

from mbrl.util.patches import circles, polygons
from mbrl.util.plot_shapes import SENSOR_POLYGON, REGULAR_OBJ_POLYGONS, REGULAR_OBJ_RADII


In [75]:
def plot_and_save_push_plots(env_kwargs, df, data_columns, trials, directory, result_type):

    loss_contact = False
    for trial in range(trials):
        fig_xy, ax = plt.subplots(figsize=(12, 6))
        ax.plot(df.query("trial==@trial")["tcp_x"], df.query("trial==@trial")["tcp_y"], "bs", label='tcp psosition')
        ax.plot(df.query("trial==@trial").query("contact==@loss_contact")["tcp_x"], df.query("trial==@trial").query("contact==@loss_contact")["tcp_y"], "g+", markersize=20)
        ax.plot(df.query("trial==@trial")["contact_x"], df.query("trial==@trial")["contact_y"], "rs", label='contact psosition')
        ax.plot(df.query("trial==@trial").query("contact==@loss_contact")["contact_x"], df.query("trial==@trial").query("contact==@loss_contact")["contact_y"], "gx", markersize=20)
        ax.plot(df.query("trial==@trial").query("goal_reached==True")['goal_x'], df.query("trial==@trial").query("goal_reached==True")['goal_y'], "x", markersize=20, markeredgewidth=3, markeredgecolor="green", label="reached goal")
        
        # If last row is goal is not reached, plot last goal
        if not df.query("trial==@trial")["goal_reached"].iloc[-1]:
            ax.plot(df.query("trial==@trial")['goal_x'].iloc[-1], df.query("trial==@trial")['goal_y'].iloc[-1], "x", markersize=20, markeredgewidth=3, markeredgecolor="black", label=" not reached goal")

        df.query("trial==@trial").query("goal_reached==True")[['goal_x', 'goal_y']]

        # Plot orn arrows
        for i, rows in df.query("trial==@trial").iterrows():
            if i % 10 == 0:
                tcp_x, tcp_y, tcp_Rz = rows["tcp_x"], rows["tcp_y"], rows["tcp_Rz"]
                tcp_dx, tcp_dy = 0.05 * np.cos(tcp_Rz), 0.05 * np.sin(tcp_Rz)
                tcp_x, tcp_y = tcp_x + tcp_dx*0.2, tcp_y + tcp_dy*0.2
                plt.arrow(tcp_x, tcp_y, tcp_dx, tcp_dy, color='b')
                obj_x, obj_y, obj_Rz= rows["contact_x"], rows["contact_y"], rows["contact_Rz"]
                obj_dx, obj_dy = 0.05 * np.cos(obj_Rz), 0.05 * np.sin(obj_Rz)
                obj_x, obj_y = obj_x + obj_dx*0.2, obj_y + obj_dy*0.2
                plt.arrow(obj_x, obj_y, obj_dx, obj_dy, color='r')
        
        
        ax.set_xlabel("x workframe")
        ax.set_ylabel("y workframe")
        ax.set_xlim([env_kwargs['env_modes']['tcp_lims'][0][0], env_kwargs['env_modes']['tcp_lims'][0][1]])
        ax.set_ylim([env_kwargs['env_modes']['tcp_lims'][1][0], env_kwargs['env_modes']['tcp_lims'][1][1]])
        ax.legend()
        fig_xy.savefig(os.path.join(directory, "workframe_plot_trial_{}.png".format(trial)))
        plt.close(fig_xy)

        fig_time_xy, axs = plt.subplots(2, 2, figsize=(14, 7.5), gridspec_kw={"width_ratios": [1, 1]})
        axs[0, 0].plot(df.query("trial==@trial")["time_steps"], df.query("trial==@trial")["tcp_x"], "bs", label='tcp ')
        axs[0, 0].plot(df.query("trial==@trial").query("contact==@loss_contact")["time_steps"], df.query("trial==@trial").query("contact==@loss_contact")["tcp_x"], "g+", markersize=20)
        axs[0, 0].plot(df.query("trial==@trial")["time_steps"], df.query("trial==@trial")["contact_x"], "rs", label='contact')
        axs[0, 0].plot(df.query("trial==@trial").query("contact==@loss_contact")["time_steps"], df.query("trial==@trial").query("contact==@loss_contact")["contact_x"], "gx", markersize=20)
        axs[0, 0].axhline(y=df.query("trial==@trial")["goal_x"].iloc[0], color='g', linestyle='-', linewidth=2)
        axs[0, 0].set_xlabel("Time steps (s)")
        axs[0, 0].set_ylabel("x axis workframe")
        axs[0, 0].set_ylim([env_kwargs['env_modes']['tcp_lims'][0][0], env_kwargs['env_modes']['tcp_lims'][0][1]])
        axs[0, 0].legend()
        axs[0, 0].grid()

        axs[0, 1].plot(df.query("trial==@trial")["time_steps"], df.query("trial==@trial")["tcp_y"], "bs", label='tcp')
        axs[0, 1].plot(df.query("trial==@trial").query("contact==@loss_contact")["time_steps"], df.query("trial==@trial").query("contact==@loss_contact")["tcp_y"], "g+", markersize=20)
        axs[0, 1].plot(df.query("trial==@trial")["time_steps"], df.query("trial==@trial")["contact_y"], "rs", label='contact')
        axs[0, 1].plot(df.query("trial==@trial").query("contact==@loss_contact")["time_steps"], df.query("trial==@trial").query("contact==@loss_contact")["contact_y"], "gx", markersize=20)
        axs[0, 1].axhline(y=df.query("trial==@trial")["goal_y"].iloc[0], color='g', linestyle='-', linewidth=2)
        axs[0, 1].set_xlabel("Time steps (s)")
        axs[0, 1].set_ylabel("y axis workframe")
        axs[0, 1].set_ylim([env_kwargs['env_modes']['tcp_lims'][1][0], env_kwargs['env_modes']['tcp_lims'][1][1]])
        axs[0, 1].legend()
        axs[0, 1].grid()

        # axs[1, 0].plot(df.query("trial==@trial")["time_steps"], df.query("trial==@trial")["goal_Rz"], "gs", label='goal Rz')
        # axs[1, 0].plot(df.query("trial==@trial")["time_steps"], df.query("trial==@trial")["contact_Rz"], "rs", label='contact Rz')
        # axs[1, 0].set_xlabel("Time steps (s)")
        # axs[1, 0].set_ylabel("contact Rz goal workframe")
        # axs[1, 0].legend()
        # axs[1, 0].grid()

        axs[1, 0].plot(df.query("trial==@trial")["time_steps"], abs(df.query("trial==@trial")["goal_Rz"] - df.query("trial==@trial")["contact_Rz"]), "gs", label='goal Rz')
        axs[1, 0].axhline(y=0, color='b', linestyle='-', linewidth=2)
        axs[1, 0].set_xlabel("Time steps (s)")
        axs[1, 0].set_ylabel("contact Rz goal workframe")
        axs[1, 0].set_ylim(-np.deg2rad(45), np.deg2rad(135))
        axs[1, 0].grid()

        axs[1, 1].plot(df.query("trial==@trial")["time_steps"], abs(df.query("trial==@trial")["contact_Rz"] - df.query("trial==@trial")["tcp_Rz"]), "bs", label='goal Rz')
        axs[1, 1].axhline(y=0, color='g', linestyle='-', linewidth=2)
        axs[1, 1].set_xlabel("Time steps (s)")
        axs[1, 1].set_ylabel("contact Rz tcp workframe")
        axs[1, 1].set_ylim(-np.deg2rad(10), np.deg2rad(30))
        axs[1, 1].grid()
        
        fig_time_xy.savefig(os.path.join(directory, "time_plot_trial_{}.png".format(trial)))
        plt.close(fig_time_xy)

In [76]:
# work_dir = r"/home/qt21590/Documents/Projects/tactile_gym_mbrl/training_model/john_reward"
# work_dir = r"/home/qt21590/Documents/Projects/tactile_gym_mbrl/training_model/stochastic_icem_H25"
work_dir = os.path.join(os.getcwd(), 'training_model')
env_kwargs_file = 'env_kwargs'
env_kwargs_dir = os.path.join(work_dir, env_kwargs_file)
env_kwargs = omegaconf.OmegaConf.load(env_kwargs_dir)

# Plot evalation results
evaluation_result_directory = os.path.join(work_dir, "evaluation_result_model_65")
data_columns = ['trial','trial_steps', 'time_steps', 'tcp_x','tcp_y','tcp_z','contact_x', 'contact_y', 'contact_z', 'goal_x', 'goal_y', 'goal_Rz', 'goal_reached', 'rewards', 'contact', 'dones']
df = pd.read_csv(os.path.join(evaluation_result_directory, 'evaluation_result.csv'))

# Make new directory for saving plots
save_directory = os.path.join(work_dir, "push_plots")
if not os.path.exists(save_directory):
    os.mkdir(save_directory)
else:
    for filename in os.listdir(save_directory):
        file_path = os.path.join(save_directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

plot_and_save_push_plots(env_kwargs, df, data_columns, 1, save_directory, 'evaluation')


In [74]:
df.query("trial==@trial")["goal_reached"].iloc[-1]

0.0